In [1]:
import numpy as np
import pandas as pd
import math
import zipfile         # a core library for working with zip files
import requests        # third-party library for making HTTP requests
pd.set_option('display.max_columns', 100)
pd.options.display.float_format = '{:.2f}'.format

# Outline

###  Part A. Sorting Households by Income Category
 1. Bring in 2018 PUMS data
 2. Assign county_id to each puma
 3. Bring in and clean, reformat county AMIs data
 4. Create dummy variables for each income category and assign to households by number of people and hh income
 5. Create count variables for households and people in each income category by  multiplying each dummy by weight variable and number of people in household

### Part B. Counting People by Age Group and Income Category
1. Create count variables for each income category and age group (under 18, adult, senior?)
2. Calculate count variable by multiplying each income category dummy by the number of people within that age category
 
### Part C. Aggregating at PUMA Level
1. Aggregate at PUMA level using groupby

## Part A1. Bring in 2018 PUMS data

In [2]:
#load zipfile from PUMS website
url = "https://www2.census.gov/programs-surveys/acs/data/pums/2018/1-Year/csv_hca.zip"
with open('csv_hca.zip', 'wb') as f:
    r = requests.get(url)
    f.write(r.content)

In [3]:
#open zipfile
z = zipfile.ZipFile('csv_hca.zip')

In [4]:
#import table to dataframe
variable_types = {"NP":"int64","NOC":"float","BDSP":"float","RMSP":"str","MV":"float","WGTP":"float","TAXAMT":"float"}
columns = ["NP","R65","NRC","PUMA","RT","BDSP","BLD","RNTP","MRGP","SMP","CONP","TEN","VACS","VALP","TAXAMT","GRPIP","GRNTP","HINCP","MV","WGTP"]
pums_df = pd.read_csv(z.open('psam_h06.csv'), 
                     low_memory=False,
                     usecols=columns)
pums_df

,RT,PUMA,WGTP,NP,BDSP,BLD,CONP,MRGP,RNTP,SMP,TEN,VACS,VALP,GRNTP,GRPIP,HINCP,MV,NRC,R65,TAXAMT
0,H,3701,0,1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,H,7306,0,1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2,H,3755,0,1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
3,H,7319,0,1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
4,H,6511,0,1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
5,H,101,0,1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
6,H,106,0,1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
7,H,8507,0,1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
8,H,5909,0,1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
9,H,5914,0,1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [5]:
#rename variables
pums_df.rename(columns={"RT":"record_type","BDSP":"number_bedrooms",
    "BLD":"units_in_structure",
    "RNTP":"monthly_rent",
    "MRGP":"first_mortgage",
    "SMP":"second_mortgage",
    "CONP":"condo_fee",
    "TEN":"tenure",
    "VACS":"vacancy_status",
    "VALP":"property_value",
    "GRPIP":"gross_rent_pct_of_income",
    "GRNTP":"gross_rent",
    "HINCP":"hh_income",
    "N65":"ppl_over_65",
    "NP":"ppl_in_hh",
    "NRC":"number_related_children",
    "WGTP":"weight",
    "TAXAMT":"prop_tax",
    "MV":"moved_in"}, inplace=True)
pums_df

,record_type,PUMA,weight,ppl_in_hh,number_bedrooms,units_in_structure,condo_fee,first_mortgage,monthly_rent,second_mortgage,tenure,vacancy_status,property_value,gross_rent,gross_rent_pct_of_income,hh_income,moved_in,number_related_children,R65,prop_tax
0,H,3701,0,1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,H,7306,0,1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2,H,3755,0,1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
3,H,7319,0,1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
4,H,6511,0,1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
5,H,101,0,1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
6,H,106,0,1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
7,H,8507,0,1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
8,H,5909,0,1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
9,H,5914,0,1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


## Part A2. Assign county id to each puma

In [42]:
# load in crosswalk file
crosswalk_df=pd.read_csv("PUMA_County_Crosswalk_v2.csv", delimiter=",")
crosswalk_df

,PUMA,county1,county2,county3,county4,county5,county6,county7
0,101,Alameda CA,NaN,NaN,NaN,NaN,NaN,NaN
1,102,Alameda CA,NaN,NaN,NaN,NaN,NaN,NaN
2,103,Alameda CA,NaN,NaN,NaN,NaN,NaN,NaN
3,104,Alameda CA,NaN,NaN,NaN,NaN,NaN,NaN
4,105,Alameda CA,NaN,NaN,NaN,NaN,NaN,NaN
5,106,Alameda CA,NaN,NaN,NaN,NaN,NaN,NaN
6,107,Alameda CA,NaN,NaN,NaN,NaN,NaN,NaN
7,108,Alameda CA,NaN,NaN,NaN,NaN,NaN,NaN
8,109,Alameda CA,NaN,NaN,NaN,NaN,NaN,NaN
9,110,Alameda CA,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
# add county name column to puma file
puma_county_df=pums_df.merge(crosswalk_df, how='left', left_on = "PUMA",right_on = "PUMA")
puma_county_df

,record_type,PUMA,weight,ppl_in_hh,number_bedrooms,units_in_structure,condo_fee,first_mortgage,monthly_rent,second_mortgage,tenure,vacancy_status,property_value,gross_rent,gross_rent_pct_of_income,hh_income,moved_in,number_related_children,R65,prop_tax,county1,county2,county3,county4,county5,county6,county7
0,H,3701,0.00,1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN
1,H,7306,0.00,1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,San Diego CA,NaN,NaN,NaN,NaN,NaN,NaN
2,H,3755,0.00,1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN
3,H,7319,0.00,1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,San Diego CA,NaN,NaN,NaN,NaN,NaN,NaN
4,H,6511,0.00,1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,Riverside CA,NaN,NaN,NaN,NaN,NaN,NaN
5,H,101,0.00,1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,Alameda CA,NaN,NaN,NaN,NaN,NaN,NaN
6,H,106,0.00,1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,Alameda CA,NaN,NaN,NaN,NaN,NaN,NaN
7,H,8507,0.00,1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,Santa Clara CA,NaN,NaN,NaN,NaN,NaN,NaN
8,H,5909,0.00,1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,Orange CA,NaN,NaN,NaN,NaN,NaN,NaN
9,H,5914,0.00,1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,Orange CA,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
# remove " CA" from end of each county name
puma_county_df['County'] = puma_county_df['county1'].str.replace(r' CA', '')
puma_county_df['county2'] = puma_county_df['county2'].str.replace(r' CA', '')
puma_county_df['county3'] = puma_county_df['county3'].str.replace(r' CA', '')
puma_county_df['county4'] = puma_county_df['county4'].str.replace(r' CA', '')
puma_county_df['county5'] = puma_county_df['county5'].str.replace(r' CA', '')
puma_county_df['county6'] = puma_county_df['county6'].str.replace(r' CA', '')
puma_county_df['county7'] = puma_county_df['county7'].str.replace(r' CA', '')
puma_county_df

,record_type,PUMA,weight,ppl_in_hh,number_bedrooms,units_in_structure,condo_fee,first_mortgage,monthly_rent,second_mortgage,tenure,vacancy_status,property_value,gross_rent,gross_rent_pct_of_income,hh_income,moved_in,number_related_children,R65,prop_tax,county1,county2,county3,county4,county5,county6,county7,County
0,H,3701,0.00,1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles
1,H,7306,0.00,1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,San Diego CA,NaN,NaN,NaN,NaN,NaN,NaN,San Diego
2,H,3755,0.00,1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles
3,H,7319,0.00,1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,San Diego CA,NaN,NaN,NaN,NaN,NaN,NaN,San Diego
4,H,6511,0.00,1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,Riverside CA,NaN,NaN,NaN,NaN,NaN,NaN,Riverside
5,H,101,0.00,1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,Alameda CA,NaN,NaN,NaN,NaN,NaN,NaN,Alameda
6,H,106,0.00,1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,Alameda CA,NaN,NaN,NaN,NaN,NaN,NaN,Alameda
7,H,8507,0.00,1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,Santa Clara CA,NaN,NaN,NaN,NaN,NaN,NaN,Santa Clara
8,H,5909,0.00,1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,Orange CA,NaN,NaN,NaN,NaN,NaN,NaN,Orange
9,H,5914,0.00,1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,Orange CA,NaN,NaN,NaN,NaN,NaN,NaN,Orange


## Part A3. Bring in and clean, reformat county AMIs data

In [45]:
#bring in 2018 county AMIs file
df = pd.read_csv("2018_AMIs.csv", delimiter=",")
df

,County,Income_Category,1,2,3,4,5,6,7,8
0,Alameda County\n4-Per,Extremely Low,24400,27900,31400,34850,37650,40450,43250,46050
1,Alameda County\n4-Per,Very Low Income,40700,46500,52300,58100,62750,67400,72050,76700
2,Alameda County\n4-Per,Low Income,62750,71700,80650,89600,96800,103950,111150,118300
3,Alameda County\n4-Per,Median Income,73100,83500,93950,104400,112750,121100,129450,137800
4,Alameda County\n4-Per,Moderate Income,87700,100250,112750,125300,135300,145350,155350,165400
5,Alpine County\n4-Pers,Extremely Low,18150,20750,23350,25900,29420,33740,38060,42380
6,Alpine County\n4-Pers,Very Low Income,30250,34600,38900,43200,46700,50150,53600,57050
7,Alpine County\n4-Pers,Low Income,46100,52650,59250,65800,71100,76350,81600,86900
8,Alpine County\n4-Pers,Median Income,66450,75900,85400,94900,102500,110100,117700,125250
9,Alpine County\n4-Pers,Moderate Income,79750,91100,102500,113900,123000,132100,141250,150350


In [46]:
#clean county names variable
df['County'] = df['County'].str.replace(r' County\n4-Person', '')
df['County'] = df['County'].str.replace(r' County\n4-Perso', '')
df['County'] = df['County'].str.replace(r' County\n4-Pers', '')
df['County'] = df['County'].str.replace(r' County\n4-Per', '')
df['County'] = df['County'].str.replace(r' County\n4-Pe', '')
df['County'] = df['County'].str.replace(r' County\n4-P', '')
df['County'] = df['County'].str.replace(r' County\n4-', '')
df['County'] = df['County'].str.replace(r' County\n4', '')
df['County'] = df['County'].str.replace(r' County\n', '')
df['County'] = df['County'].str.replace(r' County\ ' , '')
df['County'] = df['County'].str.replace(r' County', '')
df['County'] = df['County'].str.replace(r' Count', '')
df['County'] = df['County'].str.replace(r' Coun', '')
df

,County,Income_Category,1,2,3,4,5,6,7,8
0,Alameda,Extremely Low,24400,27900,31400,34850,37650,40450,43250,46050
1,Alameda,Very Low Income,40700,46500,52300,58100,62750,67400,72050,76700
2,Alameda,Low Income,62750,71700,80650,89600,96800,103950,111150,118300
3,Alameda,Median Income,73100,83500,93950,104400,112750,121100,129450,137800
4,Alameda,Moderate Income,87700,100250,112750,125300,135300,145350,155350,165400
5,Alpine,Extremely Low,18150,20750,23350,25900,29420,33740,38060,42380
6,Alpine,Very Low Income,30250,34600,38900,43200,46700,50150,53600,57050
7,Alpine,Low Income,46100,52650,59250,65800,71100,76350,81600,86900
8,Alpine,Median Income,66450,75900,85400,94900,102500,110100,117700,125250
9,Alpine,Moderate Income,79750,91100,102500,113900,123000,132100,141250,150350


In [47]:
#create df for each income level and rename variables
eli_df = df[df.Income_Category =="Extremely Low"].copy()
eli_df.rename(columns={"1":"ELI_1",
                       "2":"ELI_2",
                       "3":"ELI_3",
                       "4":"ELI_4",
                       "5":"ELI_5",
                       "6":"ELI_6",
                       "7":"ELI_7",
                       "8":"ELI_8"}, inplace=True)
eli_df.drop('Income_Category', axis=1, inplace=True)
eli_df

,County,ELI_1,ELI_2,ELI_3,ELI_4,ELI_5,ELI_6,ELI_7,ELI_8
0,Alameda,24400,27900,31400,34850,37650,40450,43250,46050
5,Alpine,18150,20750,23350,25900,29420,33740,38060,42380
10,Amador,15500,17700,20780,25100,29420,33740,38060,42380
15,Butte,13200,16460,20780,25100,29420,33740,37550,41320
20,Calaveras,15200,17400,20780,25100,29420,33740,38060,42380
25,Colusa,12600,16460,20780,25100,29420,33740,37140,39550
30,Contra Costa,24400,27900,31400,34850,37650,40450,43250,46050
35,Del Norte,12600,16460,20780,25100,29420,33740,37140,39550
40,El Dorado,16850,19250,21650,25100,29420,33740,38060,42380
45,Fresno,12600,16460,20780,25100,29420,33740,37140,39550


In [48]:
vli_df = df[df.Income_Category =="Very Low Income"].copy()
vli_df.rename(columns={"1":"VLI_1",
                       "2":"VLI_2",
                       "3":"VLI_3",
                       "4":"VLI_4",
                       "5":"VLI_5",
                       "6":"VLI_6",
                       "7":"VLI_7",
                       "8":"VLI_8"}, inplace=True)
vli_df.drop('Income_Category', axis=1, inplace=True)
vli_df

,County,VLI_1,VLI_2,VLI_3,VLI_4,VLI_5,VLI_6,VLI_7,VLI_8
1,Alameda,40700,46500,52300,58100,62750,67400,72050,76700
6,Alpine,30250,34600,38900,43200,46700,50150,53600,57050
11,Amador,25800,29450,33150,36800,39750,42700,45650,48600
16,Butte,21950,25050,28200,31300,33850,36350,38850,41350
21,Calaveras,25350,28950,32550,36150,39050,41950,44850,47750
26,Colusa,21000,24000,27000,29950,32350,34750,37150,39550
31,Contra Costa,40700,46500,52300,58100,62750,67400,72050,76700
36,Del Norte,21000,24000,27000,29950,32350,34750,37150,39550
41,El Dorado,28050,32050,36050,40050,43300,46500,49700,52900
46,Fresno,21000,24000,27000,29950,32350,34750,37150,39550


In [49]:
li_df = df[df.Income_Category =="Low Income"].copy()
li_df.rename(columns={"1":"LI_1",
                       "2":"LI_2",
                       "3":"LI_3",
                       "4":"LI_4",
                       "5":"LI_5",
                       "6":"LI_6",
                       "7":"LI_7",
                       "8":"LI_8"}, inplace=True)
li_df.drop('Income_Category', axis=1, inplace=True)
li_df

,County,LI_1,LI_2,LI_3,LI_4,LI_5,LI_6,LI_7,LI_8
2,Alameda,62750,71700,80650,89600,96800,103950,111150,118300
7,Alpine,46100,52650,59250,65800,71100,76350,81600,86900
12,Amador,41250,47150,53050,58900,63650,68350,73050,77750
17,Butte,35100,40100,45100,50100,54150,58150,62150,66150
22,Calaveras,40500,46300,52100,57850,62500,67150,71750,76400
27,Colusa,33550,38350,43150,47900,51750,55600,59400,63250
32,Contra Costa,62750,71700,80650,89600,96800,103950,111150,118300
37,Del Norte,33550,38350,43150,47900,51750,55600,59400,63250
42,El Dorado,44900,51300,57700,64100,69250,74400,79500,84650
47,Fresno,33550,38350,43150,47900,51750,55600,59400,63250


In [50]:
mi_df = df[df.Income_Category =="Median Income"].copy()
mi_df.rename(columns={"1":"MI_1",
                       "2":"MI_2",
                       "3":"MI_3",
                       "4":"MI_4",
                       "5":"MI_5",
                       "6":"MI_6",
                       "7":"MI_7",
                       "8":"MI_8"}, inplace=True)
mi_df.drop('Income_Category', axis=1, inplace=True)
mi_df

,County,MI_1,MI_2,MI_3,MI_4,MI_5,MI_6,MI_7,MI_8
3,Alameda,73100,83500,93950,104400,112750,121100,129450,137800
8,Alpine,66450,75900,85400,94900,102500,110100,117700,125250
13,Amador,51500,58900,66250,73600,79500,85400,91250,97150
18,Butte,43800,50100,56350,62600,67600,72600,77600,82650
23,Calaveras,50600,57850,65050,72300,78100,83850,89650,95450
28,Colusa,41950,47900,53900,59900,64700,69500,74300,79050
33,Contra Costa,73100,83500,93950,104400,112750,121100,129450,137800
38,Del Norte,41950,47900,53900,59900,64700,69500,74300,79050
43,El Dorado,56050,64100,72100,80100,86500,92900,99300,105750
48,Fresno,41950,47900,53900,59900,64700,69500,74300,79050


In [51]:
moi_df = df[df.Income_Category =="Moderate Income"].copy()
moi_df.rename(columns={"1":"MoI_1",
                       "2":"MoI_2",
                       "3":"MoI_3",
                       "4":"MoI_4",
                       "5":"MoI_5",
                       "6":"MoI_6",
                       "7":"MoI_7",
                       "8":"MoI_8"}, inplace=True)
moi_df.drop('Income_Category', axis=1, inplace=True)
moi_df

,County,MoI_1,MoI_2,MoI_3,MoI_4,MoI_5,MoI_6,MoI_7,MoI_8
4,Alameda,87700,100250,112750,125300,135300,145350,155350,165400
9,Alpine,79750,91100,102500,113900,123000,132100,141250,150350
14,Amador,61800,70650,79450,88300,95350,102450,109500,116550
19,Butte,52550,60100,67600,75100,81100,87100,93100,99150
24,Calaveras,60700,69400,78100,86750,93700,100650,107550,114500
29,Colusa,50350,57500,64700,71900,77650,83400,89150,94900
34,Contra Costa,87700,100250,112750,125300,135300,145350,155350,165400
39,Del Norte,87700,100250,112750,125300,135300,145350,155350,165400
44,El Dorado,67250,76900,86500,96100,103800,111500,119150,126850
49,Fresno,50350,57500,64700,71900,77650,83400,89150,94900


In [52]:
#combine dfs
all_amis_df=eli_df.merge(vli_df, how='left', left_on = "County",right_on = "County")
all_amis_df=all_amis_df.merge(li_df, how='left', left_on = "County",right_on = "County")
all_amis_df=all_amis_df.merge(mi_df, how='left', left_on = "County",right_on = "County")
all_amis_df=all_amis_df.merge(moi_df, how='left', left_on = "County",right_on = "County")
all_amis_df

,County,ELI_1,ELI_2,ELI_3,ELI_4,ELI_5,ELI_6,ELI_7,ELI_8,VLI_1,VLI_2,VLI_3,VLI_4,VLI_5,VLI_6,VLI_7,VLI_8,LI_1,LI_2,LI_3,LI_4,LI_5,LI_6,LI_7,LI_8,MI_1,MI_2,MI_3,MI_4,MI_5,MI_6,MI_7,MI_8,MoI_1,MoI_2,MoI_3,MoI_4,MoI_5,MoI_6,MoI_7,MoI_8
0,Alameda,24400,27900,31400,34850,37650,40450,43250,46050,40700,46500,52300,58100,62750,67400,72050,76700,62750,71700,80650,89600,96800,103950,111150,118300,73100,83500,93950,104400,112750,121100,129450,137800,87700,100250,112750,125300,135300,145350,155350,165400
1,Alpine,18150,20750,23350,25900,29420,33740,38060,42380,30250,34600,38900,43200,46700,50150,53600,57050,46100,52650,59250,65800,71100,76350,81600,86900,66450,75900,85400,94900,102500,110100,117700,125250,79750,91100,102500,113900,123000,132100,141250,150350
2,Amador,15500,17700,20780,25100,29420,33740,38060,42380,25800,29450,33150,36800,39750,42700,45650,48600,41250,47150,53050,58900,63650,68350,73050,77750,51500,58900,66250,73600,79500,85400,91250,97150,61800,70650,79450,88300,95350,102450,109500,116550
3,Butte,13200,16460,20780,25100,29420,33740,37550,41320,21950,25050,28200,31300,33850,36350,38850,41350,35100,40100,45100,50100,54150,58150,62150,66150,43800,50100,56350,62600,67600,72600,77600,82650,52550,60100,67600,75100,81100,87100,93100,99150
4,Calaveras,15200,17400,20780,25100,29420,33740,38060,42380,25350,28950,32550,36150,39050,41950,44850,47750,40500,46300,52100,57850,62500,67150,71750,76400,50600,57850,65050,72300,78100,83850,89650,95450,60700,69400,78100,86750,93700,100650,107550,114500
5,Colusa,12600,16460,20780,25100,29420,33740,37140,39550,21000,24000,27000,29950,32350,34750,37150,39550,33550,38350,43150,47900,51750,55600,59400,63250,41950,47900,53900,59900,64700,69500,74300,79050,50350,57500,64700,71900,77650,83400,89150,94900
6,Contra Costa,24400,27900,31400,34850,37650,40450,43250,46050,40700,46500,52300,58100,62750,67400,72050,76700,62750,71700,80650,89600,96800,103950,111150,118300,73100,83500,93950,104400,112750,121100,129450,137800,87700,100250,112750,125300,135300,145350,155350,165400
7,Del Norte,12600,16460,20780,25100,29420,33740,37140,39550,21000,24000,27000,29950,32350,34750,37150,39550,33550,38350,43150,47900,51750,55600,59400,63250,41950,47900,53900,59900,64700,69500,74300,79050,87700,100250,112750,125300,135300,145350,155350,165400
8,El Dorado,16850,19250,21650,25100,29420,33740,38060,42380,28050,32050,36050,40050,43300,46500,49700,52900,44900,51300,57700,64100,69250,74400,79500,84650,56050,64100,72100,80100,86500,92900,99300,105750,67250,76900,86500,96100,103800,111500,119150,126850
9,Fresno,12600,16460,20780,25100,29420,33740,37140,39550,21000,24000,27000,29950,32350,34750,37150,39550,33550,38350,43150,47900,51750,55600,59400,63250,41950,47900,53900,59900,64700,69500,74300,79050,50350,57500,64700,71900,77650,83400,89150,94900


## Part A4. Create dummy variables for each income category and assign to households by number of people and hh income

In [53]:
# merge datasets
hh_df=puma_county_df.merge(all_amis_df, how='left', left_on = "County",right_on = "County")
hh_df

,record_type,PUMA,weight,ppl_in_hh,number_bedrooms,units_in_structure,condo_fee,first_mortgage,monthly_rent,second_mortgage,tenure,vacancy_status,property_value,gross_rent,gross_rent_pct_of_income,hh_income,moved_in,number_related_children,R65,prop_tax,county1,county2,county3,county4,county5,county6,county7,County,ELI_1,ELI_2,ELI_3,ELI_4,ELI_5,ELI_6,ELI_7,ELI_8,VLI_1,VLI_2,VLI_3,VLI_4,VLI_5,VLI_6,VLI_7,VLI_8,LI_1,LI_2,LI_3,LI_4,LI_5,LI_6,LI_7,LI_8,MI_1,MI_2,MI_3,MI_4,MI_5,MI_6,MI_7,MI_8,MoI_1,MoI_2,MoI_3,MoI_4,MoI_5,MoI_6,MoI_7,MoI_8
0,H,3701,0.00,1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles,20350,23250,26150,29050,31400,33740,38060,42380,33950,38800,43650,48450,52350,56250,60100,64000,54250,62000,69750,77500,83700,89900,96100,102300,48500,55450,62350,69300,74850,80400,85950,91500,58200,66500,74850,83150,89800,96450,103100,109750
1,H,7306,0.00,1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,San Diego CA,NaN,NaN,NaN,NaN,NaN,NaN,San Diego,20450,23400,26300,29200,31550,33900,38060,42380,34100,38950,43800,48650,52550,56450,60350,64250,54500,62300,70100,77850,84100,90350,96550,102800,57250,65450,73600,81800,88350,94900,101450,108000,68700,78500,88350,98150,106000,113850,121700,129550
2,H,3755,0.00,1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles,20350,23250,26150,29050,31400,33740,38060,42380,33950,38800,43650,48450,52350,56250,60100,64000,54250,62000,69750,77500,83700,89900,96100,102300,48500,55450,62350,69300,74850,80400,85950,91500,58200,66500,74850,83150,89800,96450,103100,109750
3,H,7319,0.00,1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,San Diego CA,NaN,NaN,NaN,NaN,NaN,NaN,San Diego,20450,23400,26300,29200,31550,33900,38060,42380,34100,38950,43800,48650,52550,56450,60350,64250,54500,62300,70100,77850,84100,90350,96550,102800,57250,65450,73600,81800,88350,94900,101450,108000,68700,78500,88350,98150,106000,113850,121700,129550
4,H,6511,0.00,1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,Riverside CA,NaN,NaN,NaN,NaN,NaN,NaN,Riverside,14150,16460,20780,25100,29420,33740,38060,42380,23600,27000,30350,33700,36400,39100,41800,44500,37750,43150,48550,53900,58250,62550,66850,71150,46050,52650,59200,65800,71050,76350,81600,86850,55250,63150,71050,78950,85250,91600,97900,104200
5,H,101,0.00,1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,Alameda CA,NaN,NaN,NaN,NaN,NaN,NaN,Alameda,24400,27900,31400,34850,37650,40450,43250,46050,40700,46500,52300,58100,62750,67400,72050,76700,62750,71700,80650,89600,96800,103950,111150,118300,73100,83500,93950,104400,112750,121100,129450,137800,87700,100250,112750,125300,135300,145350,155350,165400
6,H,106,0.00,1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,Alameda CA,NaN,NaN,NaN,NaN,NaN,NaN,Alameda,24400,27900,31400,34850,37650,40450,43250,46050,40700,46500,52300,58100,62750,67400,72050,76700,62750,71700,80650,89600,96800,103950,111150,118300,73100,83500,93950,104400,112750,121100,129450,137800,87700,100250,112750,125300,135300,145350,155350,165400
7,H,8507,0.00,1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,Santa Clara CA,NaN,NaN,NaN,NaN,NaN,NaN,Santa Clara,27950,31950,35950,39900,43100,46300,49500,52700,46550,53200,59850,66500,71850,77150,82500,87800,66150,75600,85050,94450,102050,109600,117150,124700,87650,100150,112700,125200,135200,145250,155250,165250,105200,120200,135250,150250,162250,174300,186300,198350
8,H,5909,0.00,1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,Orange CA,NaN,NaN,NaN,NaN,NaN,NaN,Orange,23000,26250,29550,32800,35450,38050,40700,43300,38300,43750,49200,54650,59050,63400,67800,72150,61250,70000,78750,87450,94450,101450,108450,115450,64900,74150,83450,92700,100100,107550,114950,122350,77900,89000,100150,111250,120150,129050,137950,146850
9,H,5914,0.00,1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na

In [54]:
#create count variables for households in each income group

hh_df["ELI_count"]=np.where((hh_df.ppl_in_hh==1)&(hh_df.hh_income<=hh_df.ELI_1)|
                            (hh_df.ppl_in_hh==2)&(hh_df.hh_income<=hh_df.ELI_2)|
                            (hh_df.ppl_in_hh==3)&(hh_df.hh_income<=hh_df.ELI_3)|
                            (hh_df.ppl_in_hh==4)&(hh_df.hh_income<=hh_df.ELI_4)|
                            (hh_df.ppl_in_hh==5)&(hh_df.hh_income<=hh_df.ELI_5)|
                            (hh_df.ppl_in_hh==6)&(hh_df.hh_income<=hh_df.ELI_6)|
                            (hh_df.ppl_in_hh==7)&(hh_df.hh_income<=hh_df.ELI_7)|
                            (hh_df.ppl_in_hh==8)&(hh_df.hh_income<=hh_df.ELI_8),1,0)
hh_df["VLI_count"]=np.where((hh_df.ppl_in_hh==1)&(hh_df.hh_income<=hh_df.VLI_1)&(hh_df.hh_income>hh_df.ELI_1)|
                            (hh_df.ppl_in_hh==2)&(hh_df.hh_income<=hh_df.VLI_2)&(hh_df.hh_income>hh_df.ELI_2)|
                            (hh_df.ppl_in_hh==3)&(hh_df.hh_income<=hh_df.VLI_3)&(hh_df.hh_income>hh_df.ELI_3)|
                            (hh_df.ppl_in_hh==4)&(hh_df.hh_income<=hh_df.VLI_4)&(hh_df.hh_income>hh_df.ELI_4)|
                            (hh_df.ppl_in_hh==5)&(hh_df.hh_income<=hh_df.VLI_5)&(hh_df.hh_income>hh_df.ELI_5)|
                            (hh_df.ppl_in_hh==6)&(hh_df.hh_income<=hh_df.VLI_6)&(hh_df.hh_income>hh_df.ELI_6)|
                            (hh_df.ppl_in_hh==7)&(hh_df.hh_income<=hh_df.VLI_7)&(hh_df.hh_income>hh_df.ELI_7)|
                            (hh_df.ppl_in_hh==8)&(hh_df.hh_income<=hh_df.VLI_8)&(hh_df.hh_income>hh_df.ELI_8),1,0)
hh_df["LI_count"]=np.where((hh_df.ppl_in_hh==1)&(hh_df.hh_income<=hh_df.LI_1)&(hh_df.hh_income>hh_df.VLI_1)|
                           (hh_df.ppl_in_hh==2)&(hh_df.hh_income<=hh_df.LI_2)&(hh_df.hh_income>hh_df.VLI_2)|
                           (hh_df.ppl_in_hh==3)&(hh_df.hh_income<=hh_df.LI_3)&(hh_df.hh_income>hh_df.VLI_3)|
                           (hh_df.ppl_in_hh==4)&(hh_df.hh_income<=hh_df.LI_4)&(hh_df.hh_income>hh_df.VLI_4)|
                           (hh_df.ppl_in_hh==5)&(hh_df.hh_income<=hh_df.LI_5)&(hh_df.hh_income>hh_df.VLI_5)|
                           (hh_df.ppl_in_hh==6)&(hh_df.hh_income<=hh_df.LI_6)&(hh_df.hh_income>hh_df.VLI_6)|
                           (hh_df.ppl_in_hh==7)&(hh_df.hh_income<=hh_df.LI_7)&(hh_df.hh_income>hh_df.VLI_7)|
                           (hh_df.ppl_in_hh==8)&(hh_df.hh_income<=hh_df.LI_8)&(hh_df.hh_income>hh_df.VLI_8),1,0)
hh_df["MoI_count"]=np.where((hh_df.ppl_in_hh==1)&(hh_df.hh_income<=hh_df.MoI_1)&(hh_df.hh_income>hh_df.LI_1)|
                            (hh_df.ppl_in_hh==2)&(hh_df.hh_income<=hh_df.MoI_2)&(hh_df.hh_income>hh_df.LI_2)|
                            (hh_df.ppl_in_hh==3)&(hh_df.hh_income<=hh_df.MoI_3)&(hh_df.hh_income>hh_df.LI_3)|
                            (hh_df.ppl_in_hh==4)&(hh_df.hh_income<=hh_df.MoI_4)&(hh_df.hh_income>hh_df.LI_4)|
                            (hh_df.ppl_in_hh==5)&(hh_df.hh_income<=hh_df.MoI_5)&(hh_df.hh_income>hh_df.LI_5)|
                            (hh_df.ppl_in_hh==6)&(hh_df.hh_income<=hh_df.MoI_6)&(hh_df.hh_income>hh_df.LI_6)|
                            (hh_df.ppl_in_hh==7)&(hh_df.hh_income<=hh_df.MoI_7)&(hh_df.hh_income>hh_df.LI_7)|
                            (hh_df.ppl_in_hh==8)&(hh_df.hh_income<=hh_df.MoI_8)&(hh_df.hh_income>hh_df.LI_8),1,0)
hh_df["HI_count"]=np.where((hh_df.ppl_in_hh==1)&(hh_df.hh_income>hh_df.MoI_1)|
                           (hh_df.ppl_in_hh==2)&(hh_df.hh_income>hh_df.MoI_2)|
                           (hh_df.ppl_in_hh==3)&(hh_df.hh_income>hh_df.MoI_3)|
                           (hh_df.ppl_in_hh==4)&(hh_df.hh_income>hh_df.MoI_4)|
                           (hh_df.ppl_in_hh==5)&(hh_df.hh_income>hh_df.MoI_5)|
                           (hh_df.ppl_in_hh==6)&(hh_df.hh_income>hh_df.MoI_6)|
                           (hh_df.ppl_in_hh==7)&(hh_df.hh_income>hh_df.MoI_7)|
                           (hh_df.ppl_in_hh==8)&(hh_df.hh_income>hh_df.MoI_8),1,0)
hh_df

,record_type,PUMA,weight,ppl_in_hh,number_bedrooms,units_in_structure,condo_fee,first_mortgage,monthly_rent,second_mortgage,tenure,vacancy_status,property_value,gross_rent,gross_rent_pct_of_income,hh_income,moved_in,number_related_children,R65,prop_tax,county1,county2,county3,county4,county5,county6,county7,County,ELI_1,ELI_2,ELI_3,ELI_4,ELI_5,ELI_6,ELI_7,ELI_8,VLI_1,VLI_2,VLI_3,VLI_4,VLI_5,VLI_6,VLI_7,VLI_8,LI_1,LI_2,LI_3,LI_4,LI_5,LI_6,LI_7,LI_8,MI_1,MI_2,MI_3,MI_4,MI_5,MI_6,MI_7,MI_8,MoI_1,MoI_2,MoI_3,MoI_4,MoI_5,MoI_6,MoI_7,MoI_8,ELI_count,VLI_count,LI_count,MoI_count,HI_count
0,H,3701,0.00,1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles,20350,23250,26150,29050,31400,33740,38060,42380,33950,38800,43650,48450,52350,56250,60100,64000,54250,62000,69750,77500,83700,89900,96100,102300,48500,55450,62350,69300,74850,80400,85950,91500,58200,66500,74850,83150,89800,96450,103100,109750,0,0,0,0,0
1,H,7306,0.00,1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,San Diego CA,NaN,NaN,NaN,NaN,NaN,NaN,San Diego,20450,23400,26300,29200,31550,33900,38060,42380,34100,38950,43800,48650,52550,56450,60350,64250,54500,62300,70100,77850,84100,90350,96550,102800,57250,65450,73600,81800,88350,94900,101450,108000,68700,78500,88350,98150,106000,113850,121700,129550,0,0,0,0,0
2,H,3755,0.00,1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles,20350,23250,26150,29050,31400,33740,38060,42380,33950,38800,43650,48450,52350,56250,60100,64000,54250,62000,69750,77500,83700,89900,96100,102300,48500,55450,62350,69300,74850,80400,85950,91500,58200,66500,74850,83150,89800,96450,103100,109750,0,0,0,0,0
3,H,7319,0.00,1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,San Diego CA,NaN,NaN,NaN,NaN,NaN,NaN,San Diego,20450,23400,26300,29200,31550,33900,38060,42380,34100,38950,43800,48650,52550,56450,60350,64250,54500,62300,70100,77850,84100,90350,96550,102800,57250,65450,73600,81800,88350,94900,101450,108000,68700,78500,88350,98150,106000,113850,121700,129550,0,0,0,0,0
4,H,6511,0.00,1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,Riverside CA,NaN,NaN,NaN,NaN,NaN,NaN,Riverside,14150,16460,20780,25100,29420,33740,38060,42380,23600,27000,30350,33700,36400,39100,41800,44500,37750,43150,48550,53900,58250,62550,66850,71150,46050,52650,59200,65800,71050,76350,81600,86850,55250,63150,71050,78950,85250,91600,97900,104200,0,0,0,0,0
5,H,101,0.00,1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,Alameda CA,NaN,NaN,NaN,NaN,NaN,NaN,Alameda,24400,27900,31400,34850,37650,40450,43250,46050,40700,46500,52300,58100,62750,67400,72050,76700,62750,71700,80650,89600,96800,103950,111150,118300,73100,83500,93950,104400,112750,121100,129450,137800,87700,100250,112750,125300,135300,145350,155350,165400,0,0,0,0,0
6,H,106,0.00,1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,Alameda CA,NaN,NaN,NaN,NaN,NaN,NaN,Alameda,24400,27900,31400,34850,37650,40450,43250,46050,40700,46500,52300,58100,62750,67400,72050,76700,62750,71700,80650,89600,96800,103950,111150,118300,73100,83500,93950,104400,112750,121100,129450,137800,87700,100250,112750,125300,135300,145350,155350,165400,0,0,0,0,0
7,H,8507,0.00,1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,Santa Clara CA,NaN,NaN,NaN,NaN,NaN,NaN,Santa Clara,27950,31950,35950,39900,43100,46300,49500,52700,46550,53200,59850,66500,71850,77150,82500,87800,66150,75600,85050,94450,102050,109600,117150,124700,87650,100150,112700,125200,135200,145250,155250,165250,105200,120200,135250,150250,162250,174300,186300,198350,0,0,0,0,0
8,H,5909,0.00,1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,Orange CA,NaN,NaN,NaN,NaN,NaN,NaN,Orange,23000,26250,29550,32800,35450,38050,40700,43300,38300,43750,49200,54650,59050,63400,67800,72150,61250,70000,78750,87450,94450,101450,108450,115450,64900,74150,83450,92700,100100,107550,114950,122350,7790

## Part. A5. Create count variables for households and people in each income category by  multiplying each dummy by weight variable and number of people in household

In [55]:
hh_df["18_ELI_hh_count"]=hh_df.ELI_count*hh_df.weight
hh_df["18_ELI_ppl_count"]=hh_df.ELI_count*hh_df.ppl_in_hh*hh_df.weight
hh_df["18_VLI_hh_count"]=hh_df.VLI_count*hh_df.weight
hh_df["18_VLI_ppl_count"]=hh_df.VLI_count*hh_df.ppl_in_hh*hh_df.weight
hh_df["18_LI_hh_count"]=hh_df.LI_count*hh_df.weight
hh_df["18_LI_ppl_count"]=hh_df.LI_count*hh_df.ppl_in_hh*hh_df.weight
hh_df["18_MoI_hh_count"]=hh_df.MoI_count*hh_df.weight
hh_df["18_MoI_ppl_count"]=hh_df.MoI_count*hh_df.ppl_in_hh*hh_df.weight
hh_df["18_HI_hh_count"]=hh_df.HI_count*hh_df.weight
hh_df["18_HI_ppl_count"]=hh_df.HI_count*hh_df.ppl_in_hh*hh_df.weight
hh_df

,record_type,PUMA,weight,ppl_in_hh,number_bedrooms,units_in_structure,condo_fee,first_mortgage,monthly_rent,second_mortgage,tenure,vacancy_status,property_value,gross_rent,gross_rent_pct_of_income,hh_income,moved_in,number_related_children,R65,prop_tax,county1,county2,county3,county4,county5,county6,county7,County,ELI_1,ELI_2,ELI_3,ELI_4,ELI_5,ELI_6,ELI_7,ELI_8,VLI_1,VLI_2,VLI_3,VLI_4,VLI_5,VLI_6,VLI_7,VLI_8,LI_1,LI_2,LI_3,LI_4,LI_5,LI_6,LI_7,LI_8,MI_1,MI_2,MI_3,MI_4,MI_5,MI_6,MI_7,MI_8,MoI_1,MoI_2,MoI_3,MoI_4,MoI_5,MoI_6,MoI_7,MoI_8,ELI_count,VLI_count,LI_count,MoI_count,HI_count,18_ELI_hh_count,18_ELI_ppl_count,18_VLI_hh_count,18_VLI_ppl_count,18_LI_hh_count,18_LI_ppl_count,18_MoI_hh_count,18_MoI_ppl_count,18_HI_hh_count,18_HI_ppl_count
0,H,3701,0.00,1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles,20350,23250,26150,29050,31400,33740,38060,42380,33950,38800,43650,48450,52350,56250,60100,64000,54250,62000,69750,77500,83700,89900,96100,102300,48500,55450,62350,69300,74850,80400,85950,91500,58200,66500,74850,83150,89800,96450,103100,109750,0,0,0,0,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,H,7306,0.00,1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,San Diego CA,NaN,NaN,NaN,NaN,NaN,NaN,San Diego,20450,23400,26300,29200,31550,33900,38060,42380,34100,38950,43800,48650,52550,56450,60350,64250,54500,62300,70100,77850,84100,90350,96550,102800,57250,65450,73600,81800,88350,94900,101450,108000,68700,78500,88350,98150,106000,113850,121700,129550,0,0,0,0,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,H,3755,0.00,1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles,20350,23250,26150,29050,31400,33740,38060,42380,33950,38800,43650,48450,52350,56250,60100,64000,54250,62000,69750,77500,83700,89900,96100,102300,48500,55450,62350,69300,74850,80400,85950,91500,58200,66500,74850,83150,89800,96450,103100,109750,0,0,0,0,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,H,7319,0.00,1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,San Diego CA,NaN,NaN,NaN,NaN,NaN,NaN,San Diego,20450,23400,26300,29200,31550,33900,38060,42380,34100,38950,43800,48650,52550,56450,60350,64250,54500,62300,70100,77850,84100,90350,96550,102800,57250,65450,73600,81800,88350,94900,101450,108000,68700,78500,88350,98150,106000,113850,121700,129550,0,0,0,0,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,H,6511,0.00,1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,Riverside CA,NaN,NaN,NaN,NaN,NaN,NaN,Riverside,14150,16460,20780,25100,29420,33740,38060,42380,23600,27000,30350,33700,36400,39100,41800,44500,37750,43150,48550,53900,58250,62550,66850,71150,46050,52650,59200,65800,71050,76350,81600,86850,55250,63150,71050,78950,85250,91600,97900,104200,0,0,0,0,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
5,H,101,0.00,1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,Alameda CA,NaN,NaN,NaN,NaN,NaN,NaN,Alameda,24400,27900,31400,34850,37650,40450,43250,46050,40700,46500,52300,58100,62750,67400,72050,76700,62750,71700,80650,89600,96800,103950,111150,118300,73100,83500,93950,104400,112750,121100,129450,137800,87700,100250,112750,125300,135300,145350,155350,165400,0,0,0,0,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
6,H,106,0.00,1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,Alameda CA,NaN,NaN,NaN,NaN,NaN,NaN,Alameda,24400,27900,31400,34850,37650,40450,43250,46050,40700,46500,52300,58100,62750,67400,72050,76700,62750,71700,80650,89600,96800,103950,111150,118300,73100,83500,93950,104400,112750,121100,129450,137800,87700,100250,112750,125300,135300,145350,155350,165400,0,0,0,0,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
7,H,8507,0.00,1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,Santa Clara CA,NaN,NaN,NaN,NaN,NaN,NaN,Santa Clara,27950,31950,35950,39900,43100,46300,49500,52700,46550,53200,59850,66500,71850,7715

# Part C1. Aggregating at the PUMA level

In [56]:
puma_sums_df = hh_df.groupby("PUMA").sum()
puma_counts_df = puma_sums_df[["18_ELI_hh_count","18_ELI_ppl_count",
                              "18_VLI_hh_count",
                              "18_VLI_ppl_count",
                              "18_LI_hh_count",
                              "18_LI_ppl_count",
                              "18_MoI_hh_count",
                              "18_MoI_ppl_count",
                              "18_HI_hh_count",
                              "18_HI_ppl_count"]].copy()
puma_counts_df

,18_ELI_hh_count,18_ELI_ppl_count,18_VLI_hh_count,18_VLI_ppl_count,18_LI_hh_count,18_LI_ppl_count,18_MoI_hh_count,18_MoI_ppl_count,18_HI_hh_count,18_HI_ppl_count
PUMA,,,,,,,,,,
101,10607.00,23078.00,5307.00,10066.00,7057.00,14567.00,5503.00,11191.00,22498.00,56959.00
102,19598.00,36453.00,8673.00,19555.00,10252.00,21618.00,10145.00,20178.00,27063.00,57554.00
103,6068.00,8482.00,4460.00,7873.00,5523.00,11170.00,6515.00,13792.00,34548.00,84078.00
104,12957.00,29243.00,7490.00,23820.00,8713.00,26220.00,4737.00,12904.00,6711.00,20254.00
105,8547.00,18139.00,8310.00,19844.00,9659.00,23322.00,8714.00,23056.00,27672.00,77069.00
106,4432.00,8035.00,6724.00,14677.00,8536.00,26616.00,7597.00,23145.00,18018.00,56994.00
107,6955.00,16188.00,6771.00,18702.00,9760.00,30271.00,8062.00,27491.00,16597.00,53260.00
108,2312.00,5065.00,2763.00,7485.00,4286.00,13106.00,7610.00,25861.00,26536.00,85995.00
109,7027.00,13985.00,4473.00,12126.00,8387.00,22823.00,7711.00,24669.00,41261.00,124580.00


In [57]:
#export it
puma_counts_df.to_csv("2018_ppl_hh_puma_counts.csv")